# ОПИСАНИЕ ПРОЕКТА

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, вернее клиенты предлагают свои правки и комментируют изменения других. Необходим инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Задача - обучить модель классифицировать комментарии на позитивные и негативные. В наличии - набор данных с разметкой о токсичности правок.

Метрика качества *F1* должна быть не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
# Импортируем необходимые библитеки и методы
import numpy as np
import pandas as pd
import torch
import transformers
import nltk
import re
import warnings

from tqdm import notebook
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from catboost import Pool, CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

In [4]:
# Читаем исходный файл
data = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
# Смотрим общую информацию
print('INFO')
display(data.info())
print('HEAD 10')
display(data.head(10))
print('DESCRIBE')
display(data.describe())
print('SPACES')
display(data.isnull().sum())
print('DUPLICATES')
display(data.duplicated().sum())

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

HEAD 10


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


DESCRIBE


,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


SPACES


text     0
toxic    0
dtype: int64

DUPLICATES


0

Необходимо привести все записи к одному регистру

In [6]:
data['text'] = data['text'].str.lower()

Учитывая, что абсолютное большинство записей на английском языке - проведем лемматизацию с помощью `WordNetLemmatizer`, а также очистим тексты от лишних символов, с помощью регулярных выражений

In [9]:
%%time

corpus = list(data['text'])
nltk.download('punkt')
nltk.download('wordnet')

# Создаем функцию лемматизации
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])   
    return lemm_text

# Создаем функцию чистки
def clear_text(text):
    return   " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())

print("Количество текстов:", len(data['text']))
print('Проверка лемматизации')
print(" Исходный текст data[1]:", data['text'][1])
for i in range(len(data['text'])):
    data.loc[i,'text']= lemmatize(clear_text(corpus[i]))
    print("Прогресс, %:", i/159571*100)
print("Лемматизированный текст data[1]:", data['text'][1])

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Количество текстов: 159571
Проверка лемматизации
 Исходный текст data[1]: d aww he match this background colour i m seemingly stuck with thanks talk january utc
Лемматизированный текст data[1]: d aww he match this background colour i m seemingly stuck with thanks talk january utc
CPU times: user 2min 1s, sys: 436 ms, total: 2min 1s
Wall time: 2min 3s


Контрольная проверка на пропуски и дубликаты

In [10]:
print('SPACES')
display(data.isnull().sum())
print('DUPLICATES')
display(data.duplicated().sum())

SPACES


text     0
toxic    0
dtype: int64

DUPLICATES


53

Наблюдается незначительное, относительно общего размера выборки, количество дубликатов. Удалим их.

In [11]:
data = data.drop_duplicates()

Создадим обучающую, вадидационную и тестовую выборки последовательным разбиением

In [12]:
data_train, data_valid = train_test_split(data, test_size = 0.4, random_state = 12345)
data_valid, data_test = train_test_split(data_valid, test_size = 0.5, random_state = 12345)

Оценим размерность выборок

In [13]:
print("Обучающая выборка:", data_train.shape)
print("Валидная выборка:", data_valid.shape)
print("Тестовая выборка:", data_test.shape)

Обучающая выборка: (95710, 2)
Валидная выборка: (31904, 2)
Тестовая выборка: (31904, 2)


Создадим переменные для признаков и целевого признака, для каждой из выборок

In [14]:
features_train = data_train.drop(['toxic'], axis=1)
target_train = data_train['toxic']
features_valid = data_valid.drop(['toxic'], axis=1)
target_valid = data_valid['toxic']
features_test = data_test.drop(['toxic'], axis=1)
target_test = data_test['toxic']

### TF-IDF

Создадим матрицы cо значениями TF-IDF по корпусу сообщений. Укажем стоп-слова.

In [15]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Выборка features_train

In [16]:
corpus_train = list(features_train['text'])
tf_idf_train = count_tf_idf.fit_transform(corpus_train)

print("Размер матрицы:", tf_idf_train.shape)

Размер матрицы: (95710, 138499)


#### Выборка features_valid

In [17]:
corpus_valid = list(features_valid['text'])
tf_idf_valid = count_tf_idf.transform(corpus_valid)

print("Размер матрицы:", tf_idf_valid.shape)

Размер матрицы: (31904, 138499)


#### Выборка features_test

In [18]:
corpus_test = list(features_test['text'])
tf_idf_test = count_tf_idf.transform(corpus_test)

print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (31904, 138499)


## 1. Подготовка. Выводы.

- исходный датасет состоит 159571 записей с 2 признаками
- названия столбцов информативны и удобочитаемы
- типы данных признаков подходят для дальнейших расчетов
- все значения признака `text` были приведены к нижнему регистру
- в исходном датасете пробелы и дубликаты не обнаружены
- произведена лемматизация текстов. Учитывая, что все тексты признака `text` на английском языке - для лемматизации применен `WordNetLemmatizer`
- тексты очищены от лишних символов, с помощью регулярных выражений
- после лемматизации обнаружено незначительное количество дубликатов, которые были удалены
- последовательным разбиением исходного датасета созданы: обучающая (60% записей), валидационная и тестовая выборки (по 20% записей соответственно)
- созданы переменные для признаков и целевого признака каждой из выборок
- созданы матрицы со значениями `TF-IDF` и отдельные переменные для признаков каждой выборки


# 2. Обучение

### 2.1. Модель DecisionTreeClassifier

Подберем оптимальные гиперпараметры с помощью `GridSearchCV`

In [19]:
def DTC_gridsearchcv(features, target):
    model = DecisionTreeClassifier()
    param_grid = { 
        'random_state': [12345],
        'max_depth': np.arange(10, 51, 10),
        'class_weight' : ['balanced']
    }
    CV = GridSearchCV(estimator = model, param_grid = param_grid, cv = 3, scoring = 'f1')
    CV.fit(features, target)
    print('Лучшие гиперпараметры: ', CV.best_params_)
    print('Лучшая F1-мера: {:.2f}'.format(CV.best_score_))
    return CV.best_params_

#### Оптимальные параметры по TF-IDF

In [20]:
%%time
best_params_DTC_tfidf = DTC_gridsearchcv(tf_idf_train, target_train)

Лучшие гиперпараметры:  {'class_weight': 'balanced', 'max_depth': 50, 'random_state': 12345}
Лучшая F1-мера: 0.62
CPU times: user 5min 39s, sys: 740 ms, total: 5min 40s
Wall time: 5min 43s


#### Обучение модели, с оптимальными гиперпараметрами, на тренинговой выборке и оценка на валидной

In [21]:
def learning(model, features_train, target_train, features_valid, target_valid):
    '''
    Функция принимает в качестве аргументов: модель, признаки и целевой признак выборки для обучения и тестирования.
    Функция обучает заданную модель по обучающей выборке и расчитывает метрики по валидационной выборке: accuracy, F1-меру.
    '''
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print('Точность модели:{:.2%}'. format(accuracy))
    print('F1-мера: {:.2f}'. format(f1_score(target_valid, predictions)))

TF-IDF

In [22]:
%%time
model_DTC_tfidf = DecisionTreeClassifier(class_weight = 'balanced', max_depth = 50, random_state = 12345)
learning(model_DTC_tfidf, tf_idf_train, target_train, tf_idf_valid, target_valid)

Точность модели:91.73%
F1-мера: 0.62
CPU times: user 41.2 s, sys: 55.7 ms, total: 41.3 s
Wall time: 41.7 s


### 2.2. Модель LogisticRegression

Подберем оптимальные гиперпараметры с помощью GridSearchCV

In [23]:
def LR_gridsearchcv(features, target):
    model = LogisticRegression()
    param_grid = { 
        'random_state': [12345],
        'solver': ['liblinear'],
        'penalty': ['l1', 'l2'],
        'class_weight' : ['balanced']
    }
    CV = GridSearchCV(estimator = model, param_grid = param_grid, cv= 3, scoring = 'f1')
    CV.fit(features, target)
    print('Лучшие гиперпараметры: ', CV.best_params_)
    print('Лучшая F1-мера: {:.2f}'.format(CV.best_score_))
    return CV.best_params_

#### Оптимальные параметры по TF-IDF

In [24]:
%%time
best_params_LR_tfidf = LR_gridsearchcv(tf_idf_train, target_train)

Лучшие гиперпараметры:  {'class_weight': 'balanced', 'penalty': 'l1', 'random_state': 12345, 'solver': 'liblinear'}
Лучшая F1-мера: 0.76
CPU times: user 14 s, sys: 139 ms, total: 14.2 s
Wall time: 14.3 s


#### Обучение модели, с оптимальными гиперпараметрами, на тренинговой выборке и оценка на валидной

TF-IDF

In [25]:
%%time
model_LR_tfidf = LogisticRegression(class_weight  = 'balanced', penalty = 'l1', random_state = 12345, solver = 'liblinear')
learning(model_LR_tfidf, tf_idf_train, target_train, tf_idf_valid, target_valid)

Точность модели:94.23%
F1-мера: 0.75
CPU times: user 1.98 s, sys: 0 ns, total: 1.98 s
Wall time: 2 s


### 2.3. Модель RandomForestClassifier

Подберем оптимальные гиперпараметры с помощью GridSearchCV

In [26]:
def RFC_gridsearchcv(features, target):
    model = RandomForestClassifier()
    param_grid = { 
        'random_state': [12345],
        'max_depth': np.arange(10, 51, 10),
        'n_estimators' : np.arange(10, 51, 10),
        'class_weight' : ['balanced']
    }
    CV = GridSearchCV(estimator = model, param_grid = param_grid, cv= 3, scoring = 'f1')
    CV.fit(features, target)
    print('Лучшие гиперпараметры: ', CV.best_params_)
    print('Лучшая F1-мера: {:.2f}'.format(CV.best_score_))
    return CV.best_params_

#### Оптимальные параметры по TF-IDF

In [23]:
%%time
best_params_RFC_tfidf = RFC_gridsearchcv(tf_idf_train, target_train)

Лучшие гиперпараметры:  {'class_weight': 'balanced', 'max_depth': 50, 'n_estimators': 50, 'random_state': 12345}
Лучшая F1-мера: 0.48
CPU times: user 14min 8s, sys: 4.73 s, total: 14min 12s
Wall time: 14min 16s


#### Обучение модели, с оптимальными гиперпараметрами, на тренинговой выборке и оценка на валидной

TF-IDF

In [24]:
%%time
model_RFC_tfidf = RandomForestClassifier(class_weight  = 'balanced', random_state = 12345, max_depth = 50, n_estimators = 50)
learning(model_RFC_tfidf, tf_idf_train, target_train, tf_idf_valid, target_valid)

Точность модели:82.41%
F1-мера: 0.47
CPU times: user 16.4 s, sys: 77.3 ms, total: 16.5 s
Wall time: 16.5 s


Таким образом, при использованиии TF-IDF, наилучшие результаты показывает модель LogisticRegression (accuracy = 94.17%, f1 = 0.75). Проведем ее финальное тестирование на тестовой выборке.

In [25]:
%%time
predictions_LR_test = model_LR_tfidf.predict(tf_idf_test)
accuracy = accuracy_score(target_test, predictions_LR_test)
print('Точность модели:{:.2%}'. format(accuracy))
print("f1_score_test = {:.2f}".format(f1_score(target_test, predictions_LR_test)))

Точность модели:94.18%
f1_score_test = 0.75
CPU times: user 13 ms, sys: 1.42 ms, total: 14.4 ms
Wall time: 12.6 ms


Успешно. Достигнут необходимый уровень меры F1 = 0,75

# 2. Обучение. Выводы.

- проведено обучение тремя алгоритмами: DecisionTreeClassifier, LogisticRegression, RandomForestClassifier
- для борьбы с дисбалансом классов был применен гиперпараметр "balanced"
- c помощью `GridSearchCV` проведен подбор оптимальных гиперпараметров:
    - `DecisionTreeClassifier` (class_weight = 'balanced', max_depth = 50, random_state = 12345)
    - `LogisticRegression` (class_weight  = 'balanced', penalty = 'l1', random_state = 12345, solver = 'liblinear')
    - `RandomForestClassifier` (class_weight  = 'balanced', random_state = 12345, max_depth = 50, n_estimators = 50)
- по результатам обучения получены следующие результаты:
    - `DecisionTreeClassifier` (F1 = 0,66)
    - `LogisticRegression` (F1 = 0,75)
    - `RandomForestClassifier` (F1 = 0,47)
- на этапе обучения лучший результат продемонстрирован алгоритмом `LogisticRegression` (F1 = 0,75)
- на этапе финального тестирования, на тестовой выборке, алгоритм `LogisticRegression` подтвердил свою эффективность, показав необходимый, по условиям проекта, уровень меры F1 = 0,75 и accuracy = 94,18%

# 3. BERT

Принимая во внимание, что BERT очень требователен к ресурсам и как следствие времязатратен - в данном разделе мы только ознакомимся с принципом его работы, не претендуя, по результатам, на получение сколь-либо значимой меры F1, так как в задачах работы с текстом и изображениями ключевую роль играет объем выборки.
За неимением достаточных ресурсов, в виде GPU, продемонстрируем работу BERT на малой выборке - 10 000 записей из исходного датасета.

### 3.1. Подготовка

In [26]:
# Читаем исходный файл
data = pd.read_csv('/Users/Anonim/Downloads/toxic_comments.csv')

In [27]:
# Смотрим размерность
data.shape

(159571, 2)

In [28]:
# Создаем выборку из 10 000 записей
data = data.sample(10000).reset_index(drop=True)

In [29]:
data.shape

(10000, 2)

In [30]:
# Проверяем на пропуски и дубликаты
print('SPACES')
display(data.isnull().sum())
print('DUPLICATES')
display(data.duplicated().sum())

SPACES


text     0
toxic    0
dtype: int64

DUPLICATES


0

Загружаем предобученную модель и токенизатор BERT (мультиязычную версию)

In [31]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = transformers.BertModel.from_pretrained('bert-base-uncased')

Проводим токенизацию (преобразуем каждое предложение в список идентификаторов, максимальная длина = 512, но для экономии времени обучения ограничимся длиной = 8). Берем предложения очищенные от символов. Лемматизация не требуется, так как BERT понимает формы слов.

In [32]:
%%time
# add_special_token добавляет токены в начале и конце каждого предложения
tokenized = data['text'].apply((lambda x: tokenizer.encode(x[:8], add_special_tokens=True)))

CPU times: user 946 ms, sys: 2.29 ms, total: 949 ms
Wall time: 947 ms


В связи с требованиями к работе модели BERT - приводим все векторы к одной длине

In [33]:
# создаем счетчик для определения максимальной длины вектора
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
        
# приводим полученные векторы к максимальному размеру за счет прибавления
# к более коротким векторам идентификатора 0 (padding)
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
np.array(padded).shape

(10000, 10)

Указываем модели, что нули в векторах не несут значимой информации. Это необходимо для компоненты модели - attention. Отбросим эти токены и создадим "маску" для действительно важных токенов - укажем нулевые и не нулевые значения.

In [34]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10000, 10)

Используем обученный BERT для создания эмбеддингов для каждого текста по 100 текстов в батче

In [35]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
    with torch.no_grad():
        batch_embeddings = model_bert(batch, attention_mask=attention_mask_batch) 
        
    embeddings.append(batch_embeddings[0][:,0,:].numpy())

Объединяем эмбеддинги для каждого текста, которые получились по результатам

In [36]:
features_bert = np.concatenate(embeddings)

Разделяем данные на обучающую и тестовую выборки и создаем переменные для признаков

In [37]:
target_bert = data['toxic'].iloc[0:10000]
features_bert_train, features_bert_test, target_bert_train, target_bert_test = train_test_split(features_bert, target_bert, test_size = 0.25, random_state = 12345)

### 3.2. Обучение

###  Модель DecisionTreeClassifier

Подберем оптимальные гиперпараметры с помощью `GridSearchCV`

#### Оптимальные параметры по BERT

In [38]:
%%time
best_params_DTC_bert = DTC_gridsearchcv(features_bert_train, target_bert_train)

Лучшие гиперпараметры:  {'class_weight': 'balanced', 'max_depth': 20, 'random_state': 12345}
Лучшая F1-мера: 0.25
CPU times: user 48.7 s, sys: 84.8 ms, total: 48.8 s
Wall time: 48.4 s


#### Обучение модели, с оптимальными гиперпараметрами

In [39]:
%%time
model_DTC_bert = DecisionTreeClassifier(class_weight = 'balanced', max_depth = 1, random_state = 12345)
learning(model_DTC_bert, features_bert_train, target_bert_train, features_bert_test, target_bert_test)

Точность модели:52.16%
F1-мера: 0.24
CPU times: user 560 ms, sys: 1.97 ms, total: 562 ms
Wall time: 561 ms


### Модель LogisticRegression

Подберем оптимальные гиперпараметры с помощью `GridSearchCV`

#### Оптимальные параметры по BERT

In [40]:
%%time
best_params_LR_bert = LR_gridsearchcv(features_bert_train, target_bert_train)

Лучшие гиперпараметры:  {'class_weight': 'balanced', 'penalty': 'l2', 'random_state': 12345, 'solver': 'liblinear'}
Лучшая F1-мера: 0.26
CPU times: user 8min 46s, sys: 752 ms, total: 8min 47s
Wall time: 8min 43s


#### Обучение модели, с оптимальными гиперпараметрами

In [41]:
%%time
model_LR_bert = LogisticRegression(class_weight = 'balanced', penalty = 'l2', random_state = 12345, solver = 'liblinear')
learning(model_DTC_bert, features_bert_train, target_bert_train, features_bert_test, target_bert_test)

Точность модели:52.16%
F1-мера: 0.24
CPU times: user 582 ms, sys: 3.98 ms, total: 586 ms
Wall time: 589 ms


### Модель RandomForestClassifier

Подберем оптимальные гиперпараметры с помощью `GridSearchCV`

#### Оптимальные параметры по BERT

In [42]:
%%time
best_params_RFC_bert = RFC_gridsearchcv(features_bert_train, target_bert_train)

Лучшие гиперпараметры:  {'class_weight': 'balanced', 'max_depth': 10, 'n_estimators': 40, 'random_state': 12345}
Лучшая F1-мера: 0.25
CPU times: user 2min 32s, sys: 825 ms, total: 2min 33s
Wall time: 2min 34s


#### Обучение модели, с оптимальными гиперпараметрами

In [43]:
%%time
model_RFC_bert = RandomForestClassifier(class_weight = 'balanced', max_depth = 10, n_estimators = 40, random_state = 12345)
learning(model_DTC_bert, features_bert_train, target_bert_train, features_bert_test, target_bert_test)

Точность модели:52.16%
F1-мера: 0.24
CPU times: user 578 ms, sys: 2.18 ms, total: 580 ms
Wall time: 578 ms


# 3. BERT. Выводы.

К сожалению, в связи с ресурсными ограничениями, с объемом выборки в 10 000 записей не удалось достичь уровня F1 = 0,75

Как результат:

- на практике ознакомились с работой BERT
- BERT очень требователен к ресурсам
- BERT не требует предварительной лемматизации
- ключевое значение для эффективной работы алгоритма имеет объем выборки (чем больше, тем лучше)


# 4. Общий вывод

#### Цели проекта

Для исследования был представлен набор данных с разметкой о токсичности правок. Необходимо обучить модель классифицировать комментарии на позитивные и негативные, со значением метрики качества F1 не меньше 0.75.

#### Общая информация о данных

- исходный датасет состоит 159571 записей с 2 признаками
- названия столбцов информативны и удобочитаемы
- типы данных признаков подходят для дальнейших расчетов
- все значения признака `text` были приведены к нижнему регистру
- в исходном датасете пробелы и дубликаты не обнаружены
- произведена лемматизация текстов. Учитывая, что все тексты признака `text` на английском языке - для лемматизации применен `WordNetLemmatizer`
- тексты очищены от лишних символов, с помощью регулярных выражений
- после лемматизации обнаружено незначительное количество дубликатов, которые были удалены
- последовательным разбиением исходного датасета созданы: обучающая (60% записей), валидационная и тестовая выборки (по 20% записей соответственно)
- созданы переменные для признаков и целевого признака каждой из выборок
- созданы матрицы со значениями `TF-IDF` и отдельные переменные для признаков каждой выборки

#### Результаты исследования различных моделей

- проведено обучение тремя алгоритмами: DecisionTreeClassifier, LogisticRegression, RandomForestClassifier
- для борьбы с дисбалансом классов был применен гиперпараметр "balanced"
- c помощью `GridSearchCV` проведен подбор оптимальных гиперпараметров:
    - `DecisionTreeClassifier` (class_weight = 'balanced', max_depth = 50, random_state = 12345)
    - `LogisticRegression` (class_weight  = 'balanced', penalty = 'l1', random_state = 12345, solver = 'liblinear')
    - `RandomForestClassifier` (class_weight  = 'balanced', random_state = 12345, max_depth = 50, n_estimators = 50)
- по результатам обучения получены следующие результаты:
    - `DecisionTreeClassifier` (F1 = 0,66)
    - `LogisticRegression` (F1 = 0,75)
    - `RandomForestClassifier` (F1 = 0,47)
- на этапе обучения лучший результат продемонстрирован алгоритмом `LogisticRegression` (F1 = 0,75)
- на этапе финального тестирования, на тестовой выборке, алгоритм `LogisticRegression` подтвердил свою эффективность, показав необходимый, по условиям проекта, уровень меры F1 = 0,75 и accuracy = 94,17%

Также, в ознакомительных целях, было осуществлено обучение тех же моделей, но с использованием метода BERT. В связи с ресурсными ограничениями, с объемом выборки в 10 000 записей не удалось достичь уровня F1 = 0,75.

===================================================================================================================
#### Благодарю за внимание.